In [1]:
library(devtools)
load_all()

Loading required package: usethis

ℹ Loading ghostbasil

Loading required package: Matrix



In [2]:
n = 200
p = 2000
X = matrix(rnorm(n * p), n, p)
C = t(X) %*% X
beta = rnorm(p)
y = X %*% beta + rnorm(n)
y = t(X) %*% y

In [15]:
lmda = 0.1
step_size = 1 / max(eigen(C, T, T)$values) 
print(step_size)
x = rep(0, p)

[1] 0.0002909404


In [23]:
out <- solve_sub_coeffs__(C, y, lmda, step_size2, x, max_iters=1000, 1e-12)

In [24]:
print(out$beta[is.nan(out$beta)])
print(out$iters)
beta <- out$beta
nrm <- sqrt(sum(beta**2))
expected <- solve(C + lmda / nrm * diag(1, p, p), y)
print(sqrt(sum((expected-beta)**2)))

numeric(0)
[1] 75
[1] 1.145937e-07
